In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install pyreadstat
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyreadstat
import scipy.stats as stats
import matplotlib.pyplot as plt 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sav_file = "/kaggle/input/breast-cancer-survival/Breast cancer survival.sav"

df, meta = pyreadstat.read_sav(sav_file)
df


variable_labels = meta.column_labels 
variable_labels = meta.column_names_to_labels



value_labels = meta.variable_value_labels




In [ ]:
df.info()
df.describe()

variables_to_convert = ['histgrad' , 'er', 'pr', 'status']

for column in variables_to_convert:
    df[column] = df[column].astype(str).replace('nan', np.nan)

df.info()

categorical_var = ['histgrad' , 'er', 'pr', 'status']


continuous_var = ['age', 'pathsize', 'lnpos', 'time']

In [ ]:
def frequency_table(variable):
    
    unique_elements, counts = np.unique(variable.dropna(), return_counts=True)

    percentages = (counts / len(variable))  * 100
    
    value_counts_and_percentages = zip(unique_elements, counts, percentages)

    for i, j, k in value_counts_and_percentages:
        print(f"{i}: Count:  {j}, Percentage: {k:.2f}%")

    return


for var in categorical_var:
    print (f"frequency table for  {var}")
    frequency_table(df[var])
    print("Mode =", df[var].mode())
    print("#"*50)

In [ ]:
df[continuous_var].describe()


df[continuous_var].skew()
df[continuous_var].kurtosis()


concatenated_series = pd.concat([df[continuous_var].describe().T, df[continuous_var].skew()], axis=1)
concatenated_series = pd.concat([df[continuous_var].describe().T, df[continuous_var].skew().rename('skewness')], axis=1)


concatenated_series = pd.concat([
    df[continuous_var].describe().T,
    df[continuous_var].skew().rename('skewness'),
    df[continuous_var].kurtosis().rename('kurtosis')

], axis=1)

print(concatenated_series)

print('#'*50)

df['lnpos'].quantile(0.9)

In [ ]:
confidence_level = 0.95 

for var in continuous_var:
    values = df[var].dropna()
    mean = values.mean()
    std_error = stats.sem(values)
    if std_error != 0:
        lower, upper = stats.t.interval(confidence_level, len(values) - 1, loc=mean, scale=std_error)
    else:
        lower, upper = mean, mean


    concatenated_series.loc[var, 'lower_ci'] = lower
    concatenated_series.loc[var, 'upper_ci'] = upper
print(concatenated_series)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

ax[0].hist(df['lnpos'])


ax[0].set_title('Histgram for lnpos')


ax[1].hist(df['pathsize'])

ax[1].set_title('Histogram for pathsize')



In [ ]:
bin_edges = [-float('inf'), 0, float('inf')] 

bin_labels = ['NO', 'Yes']

df['lnpos_YN'] = pd.cut(df['lnpos'], bins=bin_edges, labels=bin_labels, right=True)

frequency_table(df['lnpos_YN'])

In [ ]:
bin_edges = [-float('inf'), 2, 5, float('inf')]

bin_labels = ['<= 2', '2 < <=5', '> 5']


df['pathsize_cat'] = pd.cut(df['pathsize'], bins=bin_edges, labels=bin_labels, right=True)

frequency_table(df['pathsize_cat'])

df.info()

In [ ]:
cross_tab = pd.crosstab(df['lnpos_YN'], df['status'], normalize='index')
cross_tab = pd.crosstab(df['pathsize_cat'], df['status'], normalize='index') #normalize: false/index/columns/all


cross_tab

In [ ]:
pivot_table = df.pivot_table(values='time', index='lnpos_YN', aggfunc='mean')

pivot_table = df.pivot_table(values='time', index='lnpos_YN', aggfunc=['mean', 'std'])

pivot_table = df.pivot_table(values=['time', 'age'], index='lnpos_YN', aggfunc={'time':['mean', 'std'],
                                                                               'age' :['min', 'max']})

pivot_table = df.pivot_table(values=['time'], index=['pathsize_cat', 'lnpos_YN'], aggfunc=['mean', 'std'])


pivot_table